In [12]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.styles import Font
import re

wb = openpyxl.Workbook() # Создание новой книги Excel
ws = wb.active # Присваивание активного листа

headers = ['Название', 'Страна, регион', 'Цвет', 'Сахар', 'Виноград', 'Производитель', 'Крепость', 
           'Объем', 'Регион', 'Крепость', 'Аппелласьон', 'Декантация',
           'Год', 'Потенциал хранения', 'Категория', 'Выдержка в емкости', 'Сладость',
           'Кислотность', 'Танины', 'Ароматичность', 'Тело', 'Подача', 'Гастрономия', 'Описание', 'Отзыв', 'Цена']

dict_headers = {'Название': 'A', 'Страна, регион': 'B', 'Цвет':'C', 'Сахар': 'D', 'Виноград': 'E', 'Производитель': 'F', 'Крепость': 'G', 
           'Объем': 'H', 'Регион': 'I', 'Крепость': 'J', 'Аппелласьон': 'K', 'Декантация': 'L',
           'Год': 'M', 'Потенциал хранения': 'N', 'Категория': 'O', 'Выдержка в емкости': 'P', 'Сладость': 'Q',
           'Кислотность': 'R', 'Танины': 'S', 'Ароматичность': 'T', 'Тело': 'U', 'Подача': 'V', 'Гастрономия': 'W', 'Описание': 'X', 'Отзыв': 'Y', 'Цена': 'Z'}

ws.append(headers)

bold_font = Font(bold=True) # Жирный шрифт для заголовков

for cell in ws[1]:
    cell.font = bold_font

base_url = "https://simplewine.ru"

catalog_url = base_url + "/catalog/vino/page"

response_page = requests.get(base_url)

soup_page = BeautifulSoup(response_page.text, 'html.parser')

count_row = 2 # Счетчик строк

for page in range(1, 107):
    print(page)
    
    page_url = catalog_url + f"{page}" + '/'

    response_page = requests.get(page_url)

    if response_page.status_code != 200:
        print("Ошибка при запросе к странице каталога")
        exit()

    soup_page = BeautifulSoup(response_page.text, 'html.parser')

    
    a_list = soup_page.find_all('a', class_="snippet-name js-dy-slot-click", href = True)

    for elem_a in a_list:
        url_wine = base_url + elem_a['href']
        print(url_wine)
        
        response_wine = requests.get(url_wine)

        if response_wine.status_code != 200:
            print("Ошибка при запросе к странице каталога с вином")
            exit()

        soup_wine = BeautifulSoup(response_wine.text, 'html.parser')


        url_rev = url_wine +'reviews/'
        
        response_rev = requests.get(url_rev)

        if response_rev.status_code != 200:
            print("Ошибка при запросе к странице каталога с отзывами")
            exit()

        soup_rev = BeautifulSoup(response_rev.text, 'html.parser')
                
        try:
            rev_text_list = soup_rev.find_all('div', class_ = 'product-review__item-comment')

            len_list = 0

            for rev_text in rev_text_list:
                    if rev_text.get_text(strip=True) != '':
                        len_list += 1
                        cell_address = f'{dict_headers["Отзыв"]}{count_row}'
                        ws[cell_address] = rev_text.get_text(strip=True)
                        count_row += 1

            # Название вина
            name_text = soup_wine.find('h1').get_text(strip=True)
            for count_row_ch in range(count_row - len_list, count_row):
                cell_address = f'{dict_headers["Название"]}{count_row_ch}'
                ws[cell_address] = name_text

            # Цена вина
            price_text = soup_wine.find('div', class_ = 'product-buy__old-price product-buy__with-one').get_text(strip=True)
            for count_row_ch in range(count_row - len_list, count_row):
                cell_address = f'{dict_headers["Цена"]}{count_row_ch}'
                ws[cell_address] = price_text

            #Характеристики около картинки
            ch_list = soup_wine.find_all('dt', class_ = 'product-brief__name')

            for elem_info in ch_list:

                title_ch = elem_info.get_text(strip=True)

                if title_ch == 'Виноград:':
                    links = elem_info.find_next('dd', class_='product-brief__value').find_all('a')
                    value_ch =', '.join(str(link.text) for link in links)


                else:
                    value_ch = elem_info.find_next('a', href = True).get_text(strip=True)

                col_letter = dict_headers[title_ch[:-1]]

                for count_row_ch in range(count_row - len_list, count_row):

                    # Формируем адрес ячейки
                    cell_address = f'{col_letter}{count_row_ch}'

                    # Устанавливаем значение в ячейку
                    ws[cell_address] = value_ch

            # Характеристики (основные)
            
            ch_list = soup_wine.find_all('dt', class_ = 'characteristics-params__title')

            for elem_info in ch_list:

                title_ch = elem_info.get_text(strip=True)

                if title_ch in ['Регион', 'Крепость', 'Аппелласьон', 'Год', 'Выдержка в емкости']:
                        
                    value_ch = elem_info.find_next('a', href = True).get_text(strip=True)

                else:
                    value_ch = elem_info.find_next('dd').get_text(strip=True)

                col_letter = dict_headers[title_ch[:-1]]

                for count_row_ch in range(count_row - len_list, count_row):

                    # Формируем адрес ячейки
                    cell_address = f'{col_letter}{count_row_ch}'

                    # Устанавливаем значение в ячейку
                    ws[cell_address] = value_ch

            # Сладость, Кислотность, ароматичность, тело

            ch_list = soup_wine.find_all('li', class_ = 'taste__item')

            for elem_info in ch_list:

                title_ch = elem_info.find('div', class_ = 'taste__title').get_text(strip=True)
                value_ch = len(elem_info.find_all('div', class_ = 'taste__dot taste__dot_active'))


                col_letter = dict_headers[title_ch]

                for count_row_ch in range(count_row - len_list, count_row):

                    # Формируем адрес ячейки
                    cell_address = f'{col_letter}{count_row_ch}'

                    # Устанавливаем значение в ячейку
                    ws[cell_address] = value_ch

            # Подача
            name_text = soup_wine.find('dd', class_ = 'serving__temperature-value').get_text(strip=True)
            for count_row_ch in range(count_row - len_list, count_row):
                cell_address = f'{dict_headers["Подача"]}{count_row_ch}'
                ws[cell_address] = name_text

            
            # Цвет
            name_text = soup_wine.find('a', class_ = 'page-header__catalog-category').get_text(strip=True)
            for count_row_ch in range(count_row - len_list, count_row):
                cell_address = f'{dict_headers["Цвет"]}{count_row_ch}'
                ws[cell_address] = name_text

            # Гастрономия
            gastronomy_list = soup_wine.find_all('div', class_ = 'serving__suitable-title')
                
            gast_list = []
            try:
                gast_list = []
                for a_gast_elem in gastronomy_list:
                    gast_list.append(a_gast_elem.get_text(strip=True))
                    gast_str = ', '.join(elem for elem in gast_list)

            except:
                gast_str = ''
                
            for count_row_ch in range(count_row - len_list, count_row):

                # Формируем адрес ячейки
                cell_address = f'{dict_headers["Гастрономия"]}{count_row_ch}'

                # Устанавливаем значение в ячейку
                ws[cell_address] = gast_str

            #Описание
            desk_list = soup_wine.find('ul', class_ = 'characteristics-description__list')
            description = desk_list.get_text()

            for count_row_ch in range(count_row - len_list, count_row):

                # Формируем адрес ячейки
                cell_address = f'{dict_headers["Описание"]}{count_row_ch}'

                # Устанавливаем значение в ячейку
                ws[cell_address] = description

        except:
            exit()

wb.save("simplewine_rev.xlsx")
print("Парсинг завершен. Данные сохранены в simplewine_rev.xlsx")




1
https://simplewine.ru/catalog/product/maximin_riesling_2022_075_144156/
https://simplewine.ru/catalog/product/sauvignon_blanc_sunbird_2022_075_141066/
https://simplewine.ru/catalog/product/hacienda_lopez_de_haro_hacienda_lopez_de_haro_reserva_2016_075_137306/
https://simplewine.ru/catalog/product/luigi_bosca_malbec_2021_075_139707/
https://simplewine.ru/catalog/product/shumrinka_petrikor_aligote_2019_075_137567/
https://simplewine.ru/catalog/product/crab_more_zinfandel_075_141328/
https://simplewine.ru/catalog/product/sikory_gerts_2018_075_137179/
https://simplewine.ru/catalog/product/vellodoro_pecorino_2022_075_142368/
https://simplewine.ru/catalog/product/bourgogne_jurassique_2022_075_141626/
https://simplewine.ru/catalog/product/sauvignon_blanc_2022_075_138863/
https://simplewine.ru/catalog/product/vermentino_di_toscana_2022_075_143374/
https://simplewine.ru/catalog/product/les_sarrins_2022_075_142993/
https://simplewine.ru/catalog/product/colterenzio_pfefferer_2022_075_140066/
ht

: 